In [11]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

requestsSession = rq.Session()
loginUrl = "https://www.worldpulse.com/en/my-pulse/login"
resp = requestsSession.get(loginUrl)
soup = BeautifulSoup(resp.content, 'html.parser')
main = soup.select("main")[0]
authToken = [inpFields['value'] for inpFields in main.find(id="user-login").find_all("input")
                if inpFields['name'] == 'form_build_id']

auth = {
    "name": "dineshat@gmail.com",
    "pass": "mdkumar84",
    "form_build_id": authToken[0],
    "form_id": "user_login"
}
#auth["name"] = "dineshat@gmail.com"
#auth["pass"] = "mdkumar84"
#auth["form_build_id"] = authToken[0]
#auth["form_id"] = "user_login"

resp = requestsSession.post(loginUrl, data = auth, headers = dict(referer = loginUrl))
#print(resp.status_code)
#print(resp.url)
#print(resp.text)

dfFinal = pd.DataFrame({
            "Date" : [],
            "UserId" : [],
            "UserName" : [],
            "Country" : [],
            "Loves" : [],
            "Encourages" : [], 
        "topicLinks" : [] 
        })

for counter in range(1501, 1575):
    if counter == 0:
        url = "https://www.worldpulse.com/en/search?search_term="
    else:
        url = "https://www.worldpulse.com/en/search?search_term=&sort_bef_combine=created%20"\
                "DESC&sort_order=DESC&sort_by=created&page=" + str(counter)
    resp = requestsSession.get(url, headers = dict(referer = url))
    soup = BeautifulSoup(resp.content, 'html.parser')
    main = soup.find("main")
    
    authorDates = [author.find(class_="node-date").get_text() 
                   if author.find(class_="node-date") != None else '' 
                   for author in main.find_all(class_="views-row")
                  ]
    authorIds = [author.find(class_="profile-picture")['href'] 
                 if author.find(class_="profile-picture") != None else ''
                   for author in main.find_all(class_="views-row")
                ]
    authorNames = [author.find_all("a")[1].get_text() 
                   if len(author.find_all("a")) >= 2 else '' 
                   for author in main.find_all(class_="views-row")
                  ]
    authorCountries = [author.find_all("img")[1]['title'] 
                       if len(author.find_all("img")) >= 2 else '' 
                       for author in main.find_all(class_="views-row")
                      ]
    sendMeLoves = [author.find(class_="num-count").get_text() 
                   if author.find(class_="num-count") != None else 0 
                   for author in main.find_all(class_="views-row")
                  ]
    encourageMes = [author.find(class_="encourage-me").get_text() 
                    if author.find(class_="encourage-me") != None else 0 
                    for author in main.find_all(class_="views-row")
                   ]
    topicLinks = [author.select("h3 a")[0]['href'] for author in main.find_all(class_ = "views-row")]
    df = pd.DataFrame({
            "Date" : authorDates,
            "UserId" : authorIds,
            "UserName" : authorNames,
            "Country" : authorCountries,
            "Loves" : sendMeLoves,
            "Encourages" : encourageMes, 
            "topicLinks" : topicLinks
        })
    frames = [dfFinal, df]
    dfFinal = pd.concat(frames)
    if counter > 0 and counter % 250 == 0:
        dfFinal.to_csv('c:\\users\\muralid\\Documents\\wpcommunitydata\\wpCommunity_' + str(counter) + '.csv', index=False)
        dfFinal = pd.DataFrame({
            "Date" : [],
            "UserId" : [],
            "UserName" : [],
            "Country" : [],
            "Loves" : [],
            "Encourages" : [], 
                "topicLinks" : [] 
        })
        
dfFinal.to_csv('c:\\users\\muralid\\Documents\\wpcommunitydata\\wpCommunity_' + str(counter) + '.csv', index=False)



